In [1]:
# Higgs Production at LO and NLO
#
# Author: Hao Chen 陈豪 (Zhejiang University)
#
# Reference: 
# e.g. Page 2-4 of https://arxiv.org/pdf/hep-ph/0306211.pdf
# Catani, S., de Florian, D., Grazzini, M. and Nason, P., 2003. Soft-gluon resummation for Higgs boson production at hadron colliders. Journal of High Energy Physics, 2003(07), p.028.

In [2]:
import numpy as np
import scipy.integrate as integrate

In [3]:
# unit conversion: 1 picobarn = 2.56819e-9 GeV
one_pb = 2.56819e-9

In [4]:
# Constants

# color factors
nf = 5; CF = 4/3;CA = 3;TF = 1/2;
zeta3 = 1.20205690315959; # zeta function value zeta(3) 

# QCD beta function constants
beta0 = (11*CA)/3 - (4*nf*TF)/3
beta1 = (34*CA**2)/3 - (20*CA*nf*TF)/3 - 4*CF*nf*TF
beta2 = (2857*CA**3)/54 + ((-1415*CA**2)/27 - (205*CA*CF)/9 + 2*CF**2)*nf*TF + ((158*CA)/27 + (44*CF)/9)*nf**2*TF**2
beta3 = 149753/6 + (1093*nf**3)/729 + nf*(-1078361/162 - (6508*zeta3)/27) + 3564*zeta3 + nf**2*(50065/162 + (6472*zeta3)/81)
Beta1 = beta1/beta0; Beta2 = beta2/beta0; Beta3 = beta3/beta0;

mu0 = 10; lambda_QCD = 213/1000; ll0 = 2*np.log(mu0/lambda_QCD);

# Running coupling a_s = alpha_s/4*pi
def a_s(mu):
    t = 2*np.log(mu) - 2*np.log(mu0) 
    ll = t + ll0 
    LL = beta0 * ll
    return (1 + Beta3/(2*LL**3) + Beta2/LL**2 - (3*Beta1*Beta2*np.log(ll))/LL**3 - (Beta1*np.log(ll))/LL + (Beta1**2*(-1 - np.log(ll) + np.log(ll)**2))/LL**2 + (Beta1**3*(-1/2 + 2*np.log(ll) + (5*np.log(ll)**2)/2 - np.log(ll)**3))/LL**3)/LL

alpha_s = lambda mu: a_s(mu) * 4 * np.pi

In [5]:
# Parameters
m_H = 125 # Higgs mass
m_t = 162.7 # top quark mass
m_b = 4.18 # b quark mass

# Fermi constant
G_F = 1.16639e-5

# scale setting
mu_R = m_H/2 # scale for alpha_s
mu_F = m_H/2 # scale for Parton Distribution Function (PDF)


# Born level constant definition
# see eq. (3) (4) in the reference
def A_Q(x):
    if x>= 1:
        return 3*x/2*(1 + (1 - x) * np.arcsin(1/np.sqrt(x))**2)
    else:
        return 3*x/2*(1 + (1 - x) * (-1/4) * (np.log((1+np.sqrt(1-x))/(1-np.sqrt(1-x)))+np.pi * 1j)**2)

sigma_0 = G_F/(288 * np.pi * np.sqrt(2)) * abs(A_Q(4*(m_t/m_H)**2) + A_Q(4*(m_b/m_H)**2) )**2

# sigma_0 in the unit of picobarn
sigma_0_pb = sigma_0/one_pb

Kinematic variables: 

$z=m_h^2/\hat{s}$ where $\hat{s}$ is the partonic center-of-mass energy squared

$\tau = m_h^2/s$ with $s$ being the hadronic center-of-mass energy squared

Relation of hadronic cross section $\sigma(\tau)$ to partonic cross section $\hat{\sigma}(z, \alpha_s(\mu_R), \mu_F)$ via 
parton distribution functions $f_i(x, \mu_F)$:

$$\sigma(\tau) = \sum_{i,j} \tau \int_0^1 \frac{dx}{x}\frac{dz}{z} f_i(x, \mu_F)f_j\left(\frac{\tau}{x z}, \mu_F \right) \frac{\hat{\sigma}_{ij}(z, \alpha_s(\mu_R), \mu_F)}{z}\,.$$

In the following, we will use
$$\sigma^{(0)} G_{ij}(z, \alpha_s(\mu_R), \mu_F) = \frac{1}{z}\hat{\sigma}(z, \alpha_s(\mu_R), \mu_F)\, .$$

Partonic function $G_{ij}$ has the perturbative expansion w.r.t. strong coupling constant $\alpha_s(\mu_R)$:
$$ G_{ij} = \alpha_s^2(\mu_R) \left(G_{ij}^{(0)} + \frac{\alpha_s(\mu_R)}{\pi} G_{ij}^{(1)} + \cdots\right). $$

In [6]:
# load pdf with LHAPDF
import lhapdf
p = lhapdf.mkPDF("CT10nlo", 0)
p = lhapdf.mkPDF("CT10nlo/0")

# Usage:
# e.g.
# for pid in p.flavors():
#    print(p.xfxQ(pid, 0.01, 91.2))

# Partical ID
# pid = 21 is gluon
# pid = (-)n is a(n) (anti)-quark with n = 1,2,3,4,5

# Note: 
# xfxQ(i, x, Q) is not PDF f_i(x, Q) itself, it is PDF times the momentum fraction x*f_i(x, Q)

We define the luminosity kernel function that only involves PDF:
$$ 
dx\, dz\, K_{ij} (\tau, x, z, \mu_F)= dx\, dz\, \frac{1}{x} \left[ x\, f_i(x, \mu_F)\right]\left[\frac{\tau}{x z} f_{j}\left(\frac{\tau}{x z}, \mu_F\right)\right]\,.
$$
The function $x\, f_i(x,\mu_F)$ is accessible through interface function `xfxQ(i, x, mu_F)` in LHAPDF.

Note that PDF $f_i(x, \mu_F)$ contains the Heaviside function $\Theta(0 \leq x\leq 1)$.

In [7]:
# luminosity functions
def kernel_gg(tau, x, z, mu_F):
    x2 = tau/x/z
    if x2 < 1:
        return  p.xfxQ(21, x, mu_F) * p.xfxQ(21, x2, mu_F)/x
    else:
        return 0
    
def kernel_qg(tau, x, z, mu_F):
    x2 = tau/x/z
    if x2 < 1:
        q_pdf_sum_1 = sum([p.xfxQ(n, x, mu_F) for n in range(1,6)]) + sum([p.xfxQ(n, x, mu_F) for n in range(-5,0)])
        q_pdf_sum_2 = sum([p.xfxQ(n, x2, mu_F) for n in range(1,6)]) + sum([p.xfxQ(n, x2, mu_F) for n in range(-5,0)])
        return (p.xfxQ(21, x, mu_F) * q_pdf_sum_2 + q_pdf_sum_1 * p.xfxQ(21, x2, mu_F) )/x
    else:
        return 0
    
def kernel_qqb(tau, x, z, mu_F):
    x2 = tau/x/z
    if x2 < 1:
        return  (
            sum([p.xfxQ(n, x, mu_F) * p.xfxQ(-n, x2, mu_F) for n in range(1,6)]) + sum([p.xfxQ(n, x, mu_F) * p.xfxQ(-n, x2, mu_F) for n in range(-5,0)])
        )/x
    else:
        return 0

At LO, $G_{ij}^{(0)}=\delta_{i g}\delta_{j g} \delta(1-z)$, we have
$$
\sigma^{\mathrm{LO}}(\tau) = \sigma^{(0)} \alpha_s(\mu_R)^2 \int_{0}^{1} dx\, K_{gg}(\tau, x, z = 1, \mu_F) 
$$

In [8]:
# LO cross section
def sigma_LO(Q, mu_F, mu_R):
    tau = (m_H/Q)**2
    return sigma_0_pb * (alpha_s(mu_R))**2 * integrate.quad(lambda x:  kernel_gg(tau, x, 1, mu_F), 0, 1, maxp1=200, limit=200)[0]

In [9]:
# LO result (13 TeV)
LO_result = sigma_LO(13*1000, mu_F, mu_R)
print(LO_result)

14.878355679340281


In [10]:
# NLO functions
# can be found in eq. (8)-(10) in the given reference
P_gg_reg = lambda z: 6 * (1/z - 2 + z*(1-z))
P_gq = lambda z: 4/3*(1 + (1-z)**2)/z

# gg delta function part
def G1_gg_delta(mu_F, mu_R):
    return 11/2 + np.pi**2 + (33 - 2 * nf)/6 * 2 * np.log(mu_R/mu_F)
# gg plus distribution part
def G1_gg_plus(z, mu_F, mu_R):
    return 12 * np.log(1-z)/(1-z) + 6 * 2 * np.log(m_H/mu_F)/(1 - z)
# gg normal function part
def G1_gg_normal(z, mu_F, mu_R):
    return P_gg_reg(z) * np.log(((1-z)*m_H/mu_F)**2 /z) - 6*np.log(z)/(1-z) - 11/2*(1-z)**3 /z

def G1_gq_normal(z, mu_F, mu_R):
    return 1/2*P_gq(z)*np.log(((1-z)*m_H/mu_F)**2 /z) + 2/3*z - (1-z)**2/z

def G1_qqb_normal(z, mu_F, mu_R):
    return 32/27*(1-z)**3/z

In [11]:
# NLO cross section
def sigma_NLO(Q, mu_F, mu_R):
    tau = (m_H/Q)**2
    gg_integral_delta = integrate.quad(lambda x:  kernel_gg(tau, x, 1, mu_F), tau, 1, maxp1=200, limit=200)[0] * G1_gg_delta(mu_F, mu_R)
    gg_integral_plus = integrate.dblquad(lambda x, z: (kernel_gg(tau, x, z, mu_F) - kernel_gg(tau, x, 1, mu_F)) * G1_gg_plus(z, mu_F, mu_R), 0, 1, lambda x:0, lambda x:1 , epsabs=1e-6,epsrel=1e-4)[0]
    gg_integral_normal = integrate.dblquad(lambda x, z: kernel_gg(tau, x, z, mu_F) * G1_gg_normal(z, mu_F, mu_R), 0, 1, lambda x:0, lambda x:1 ,epsabs=1e-6, epsrel=1e-4)[0]
    
    gq_integral_normal = integrate.dblquad(lambda x, z: kernel_qg(tau, x, z, mu_F) * G1_gq_normal(z, mu_F, mu_R), 0, 1, lambda x:0, lambda x:1 ,epsabs=1e-6, epsrel=1e-4)[0]
    
    qqb_integral_normal = integrate.dblquad(lambda x, z: kernel_qqb(tau, x, z, mu_F) * G1_qqb_normal(z, mu_F, mu_R), 0, 1, lambda x:0, lambda x:1 ,epsabs=1e-6, epsrel=1e-4)[0]
    return alpha_s(mu_R)**3/np.pi * sigma_0_pb * (gg_integral_delta + gg_integral_plus + gg_integral_normal + gq_integral_normal + qqb_integral_normal)

In [12]:
# NLO corrections (13 TeV)
NLO_corrections = sigma_NLO(13*1000, mu_F, mu_R)
print(NLO_corrections)

19.359977796449087


In [13]:
# total cross section = LO result + NLO corrections (13 TeV)
total_Xsec = LO_result + NLO_corrections
print(total_Xsec)

34.23833347578937
